<a href="https://colab.research.google.com/github/ssgalitsky/PedalNetRT/blob/master/colab_TPU_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



links:

PyTorch Lightning Documentation https://pytorch-lightning.readthedocs.io/en/stable/

In [1]:
#@title 1. clone  https://github.com/ssgalitsky/PedalNetRT.git
!git clone https://github.com/ssgalitsky/PedalNetRT.git

Cloning into 'PedalNetRT'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 178 (delta 86), reused 27 (delta 3), pack-reused 0
Receiving objects: 100% (178/178), 56.20 MiB | 15.91 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
#@title 2.1 install dependencies
!pip3 install torchvision
!pip3 install pytorch_lightning==0.9.0
use_tpu=False

In [ ]:
#@title 2.2 install optional dependencies (TPU)
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python3 pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!pip3 install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl
use_tpu=True

In [4]:
import torch
use_cuda=True
if use_cuda and torch.cuda.is_available():
  net.cuda()

#from google.colab import drive
#drive.mount('/content/drive')

import os
os.chdir('/content/PedalNetRT')

In [ ]:
#@title 3. prepare data
!python3 "prepare_data.py" "/content/PedalNetRT/data/guitar.wav" "/content/PedalNetRT/data/voice.wav"


In [12]:
#@title 4. **train model** Beware, checkpoint files not visible via Colab webui (-rw-r--r--) 
import os
if use_tpu:
  assert os.environ['COLAB_TPU_ADDR']
  
import pytorch_lightning
!python3 "train.py" --batch_size=32 --max_epochs=1000 --learning_rate=3e-3 --num_channels=4 
#!python3 "train.py" --batch_size=32 --max_epochs=100 --learning_rate=3e-3 --num_channels=5 






2020-10-14 15:35:26.338933: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
GPU available: False, used: False
TPU available: True, using: 8 TPU cores
training on 8 TPU cores
INIT TPU local core: 0, global rank: 0 with XLA_USE_BF16=None
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)
INIT TPU local core: 6, global rank: 6 with XLA_USE_BF16=None
INIT TPU local core: 5, global rank: 5 with XLA_USE_BF16=None
INIT TPU local core: 2, global rank: 2 with XLA_USE_BF16=None
INIT TPU local core: 1, global rank: 1 with XLA_USE_BF16=None
INIT TPU local core: 3, global rank: 3 with XLA_USE_BF16=None
INIT TPU local core: 7, global rank: 7 with XLA_USE_BF16=None
INIT TPU local core: 4, global rank: 4 with XLA_USE_BF16=None

  |

In [14]:
!ls -lra lightning_logs/version_1/checkpoints/
!ls -lra /content/PedalNetRT/lightning_logs/version_0/checkpoints/epoch=1370.ckpt
#!chmod -R 755 lightning_logs 


total 64
-rw-r--r-- 1 root root 14959 Oct 14 16:05 'epoch=711.tmp_end.ckpt'
-rw-r--r-- 1 root root 40687 Oct 14 15:57 'epoch=711.ckpt'
drwxr-xr-x 3 root root  4096 Oct 14 15:35  ..
drwxr-xr-x 2 root root  4096 Oct 14 16:05  .
-rw-r--r-- 1 root root 52015 Oct 14 15:04 '/content/PedalNetRT/lightning_logs/version_0/checkpoints/epoch=1370.ckpt'


In [13]:
#@title 4. train.py
import pytorch_lightning as pl
from model import PedalNet


def trainFit(args):
    model = PedalNet(args)
    trainer = pl.Trainer(
        #max_epochs=args.max_epochs, gpus=args.gpus, row_log_interval=100
        # The following line is for use with the Colab notebook when training on TPUs.
        # Comment out the above line and uncomment the below line to use.
        
         max_epochs=args.max_epochs, tpu_cores=args.tpu_cores, row_log_interval=100
    )
    trainer.fit(model)

    parser.add_argument("--num_channels", type=int, default=5)
    parser.add_argument("--dilation_depth", type=int, default=10)
    parser.add_argument("--num_repeat", type=int, default=1)
    
    # filter_width=kernel_size
    parser.add_argument("--kernel_size", type=int, default=3)

    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--learning_rate", type=float, default=3e-3)

    parser.add_argument("--max_epochs", type=int, default=100)
    parser.add_argument("--gpus", default="0")
    parser.add_argument("--tpu_cores", default="8")

    parser.add_argument("--data", default="data.pickle")
    args = parser.parse_args()
    main(args)

In [15]:
#@title 5. convert torch model (ckpt) to plugin model (json) 
# The .ckpt model must be converted to a .json model to run in the plugin. Usage:

#!python3 convert_pedalnet_to_wavnetva.py --model=your_trained_model.ckpt
!python3 convert_pedalnet_to_wavenetva.py --model=/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=711.ckpt





2020-10-14 16:08:29.643175: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [17]:
!python test.py --model=/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=711.ckpt

2020-10-14 16:09:52.906032: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [18]:
!python plot_wav.py

Error to signal (with pre-emphasis filter):  1.0733798
Error to signal (no pre-emphasis filter):  1.0733795
Creating spectrogram data..
plot_wav.py:116: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(times, frequencies, 10*np.log10(spectrogram))
<Figure size 1300x800 with 3 Axes>
<Figure size 1200x800 with 2 Axes>


In [ ]:
#@title 6. convert model from TPU to CPU format
import torch_xla
import torch_xla.core.xla_model as xm

from model import PedalNet
# Change path below to match model file
model = PedalNet.load_from_checkpoint('lightning_logs/version_0/checkpoints/epoch=1.ckpt')

# xm.save(model.state_dict(), 'tpu_to_cpu.ckpt') 
xm.save(model, 'tpu_to_cpu_model.ckpt') 
# issues loading model, see pytorch_lightning issues #2303 and #3044 (Might be completed by now)

# TODO: Add code to be able to load saved model sucessfully
